In [2]:
import pandas as pd
import json
import re
import random
import requests
from bs4 import BeautifulSoup
from fuzzywuzzy import process, fuzz
from fractions import Fraction
from faker import Faker

In [3]:
file_path = "./datasets_for_generator"
pd.set_option('display.max_colwidth', None)

!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!


NATIONALITIES !!!!!!!!!!!!!!!!!!!!!!!


!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

In [46]:
def load_and_prepare_data():
    
    flags_df = pd.read_csv(file_path + 'flags2.csv', delimiter=',', encoding='ISO-8859-1')
    countries_df = pd.read_csv(file_path + 'countries.csv')

    flags_df.rename(columns={'Country': 'Name', 'Flag_image_url': 'image'}, inplace=True)

    merged_df = pd.merge(flags_df[['Name', 'image']], countries_df[['Name', 'Nationality']], on='Name', how='inner')
    
    merged_df = merged_df[merged_df['image'].notna() & (merged_df['image'] != '')]
    
    new_entries = pd.DataFrame({
        'Name': ['United Kingdom', 'United States'],
        'image': [
            'https://upload.wikimedia.org/wikipedia/commons/thumb/a/a5/Flag_of_the_United_Kingdom_%281-2%29.svg/383px-Flag_of_the_United_Kingdom_%281-2%29.svg.png',
            'https://upload.wikimedia.org/wikipedia/commons/thumb/a/a9/Flag_of_the_United_States_%28DoS_ECA_Color_Standard%29.svg/383px-Flag_of_the_United_States_%28DoS_ECA_Color_Standard%29.svg.png'
        ],
        'Nationality': ['British', 'American']
    })

    merged_df = pd.concat([merged_df, new_entries], ignore_index=True)
    
    return merged_df

nationality_dict = {}
def generate_sql(merged_data):
    sql_statements = []
    image_id = 1 

    for _, row in merged_data.iterrows():
        image_sql = f"""INSERT INTO Image (image_url, description) VALUES ('{row['image']}', 'This is the flag of {row['Name']}');"""
        sql_statements.append(image_sql)

        nationality_sql = f"""INSERT INTO Nationality (name, image_id) VALUES ('{row['Nationality']}', {image_id});"""
        sql_statements.append(nationality_sql)
        
        nationality_dict[image_id] = row['Nationality']

        image_id += 1 

    return sql_statements, image_id - 1 

merged_data = load_and_prepare_data()

sql_commands, last_nationality_id = generate_sql(merged_data)
last_image_id = last_nationality_id

with open(file_path + 'insert_data.sql', 'w', encoding='utf-8') as file:
    file.write('\n'.join(sql_commands))
    file.write('\n')

print("SQL generation complete. All insertion queries for images and nationalities have been saved.")
print(f"Last nationality ID used (total nationalities): {last_nationality_id}")

SQL generation complete. All insertion queries for images and nationalities have been saved.
Last nationality ID used (total nationalities): 178


!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

FOOD GROUPS!!!!!!!!!!!!!

!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

In [5]:
food_group_dict = {
    1: 'Spices and essential oils',
    2: 'Coffee, tea and their products',
    3: 'Preserved foods',
    4: 'Sweeteners',
    5: 'Fats and oils',
    6: 'Milk, eggs and their products',
    7: 'Meat and its products',
    8: 'Fish and their products',
    9: 'Cereals and their products',
    10: 'Various foods of plant origin',
    11: 'Products with sweeteners',
    12: 'Various drinks'
}

In [47]:
image_urls = [
    "https://www.ingredientsnetwork.com/47/product/127/23/83/herbs-and-spices.jpg",
    "https://imageio.forbes.com/specials-images/imageserve/6393aebacd1e996dc7b13be0/Herbal-tea-and-espresso-coffee/0x0.jpg?format=jpg&crop=1325,993,x81,y0,safe&width=1440",
    "https://thehouseandhomestead.com/wp-content/uploads/2019/07/Preserve-the-Harvest-Feature-Photo-2-1024x683.jpg",
    "https://avitahealth.org/wp-content/uploads/2022/03/sweeteners-feature.jpg",
    "https://mistafood.com/wp-content/uploads/2022/05/fats-oils-spreads-1200x675.jpeg",
    "https://samsungfood.com/wp-content/uploads/2023/02/shutterstock_1679020255.jpg",
    "https://images.ctfassets.net/3s5io6mnxfqz/5GlOYuzg0nApcehTPlbJMy/140abddf0f3f93fa16568f4d035cd5e6/AdobeStock_175165460.jpeg?w=828",
    "https://www.qld.gov.au/__data/assets/image/0022/167422/varieties/Aggregation.jpg",
    "https://fittify.in/cdn/shop/articles/4_adc6f48f-de12-4132-88ed-8a04805b8e4b.jpg?v=1673267293&width=1000",
    "https://blogassets.vita4you.gr/blog-vita4you/wp-content/uploads/2023/01/%CE%A6%CF%85%CF%84%CE%B9%CE%BA%CE%AD%CF%82-%CF%80%CF%81%CF%89%CF%84%CE%B5%CE%90%CE%BD%CE%B5%CF%82-768x512.png",
    "https://additive-free-living.s3.ap-southeast-2.amazonaws.com/wp-content/uploads/2023/07/11123357/Artificial-Sweeteners-800x533.jpg",
    "https://fireandiceontobycreek.com/wp-content/uploads/2020/11/various-cocktail-drinks.jpg"
]

sql_commands = []

food_groups = [
    'Spices and essential oils', 'Coffee, tea and their products', 'Preserved foods',
    'Sweeteners', 'Fats and oils', 'Milk, eggs and their products', 'Meat and its products',
    'Fish and their products', 'Cereals and their products', 'Various foods of plant origin',
    'Products with sweeteners', 'Various drinks'
]

for idx, url in enumerate(image_urls, start=1):
    last_image_id += 1
    description = f"""This is an image for the food group {food_groups[idx-1]}."""
    image_sql = f"""INSERT INTO Image (image_url, description) VALUES ('{url}', '{description}');"""
    sql_commands.append(image_sql)


descriptions = [
    'Essential for flavoring and seasoning a variety of dishes.',
    'Includes all varieties of coffee and tea products.',
    'Foods processed to ensure longer shelf life.',
    'Natural and artificial substances used to sweeten food.',
    'Essential cooking mediums and flavor enhancers.',
    'Dairy products and eggs used in a multitude of recipes.',
    'All types of meat products including processed meats.',
    'Includes all fish and seafood items.',
    'Staple ingredients derived from various grains.',
    'Encompasses a diverse range of plant-based foods.',
    'Foods specifically containing added sweeteners.',
    'Includes a variety of beverages.'
]

identities = ['Spice', 'Beverage', 'Preserved', 'Sweetener', 'Fat', 'Dairy', 'Meat', 'Seafood', 'Cereal', 'Plant-Based', 'Sweetened', 'Drink']

for idx, (group, desc, identity) in enumerate(zip(food_groups, descriptions, identities), start=1):
    group_sql = f"""INSERT INTO Food_Group (name, description, image_id, group_identity) VALUES ('{group}', '{desc}', {last_image_id - 12 + idx}, '{identity}');"""
    sql_commands.append(group_sql)

with open(file_path + 'insert_data.sql', 'a', encoding='utf-8') as file:
    file.write('\n'.join(sql_commands))
    file.write('\n')

print("All insertion queries for images and food groups have been saved to 'insert_data.sql'.")


All insertion queries for images and food groups have been saved to 'insert_data.sql'.


!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

EQUIPMENET !!!!!!!!!!!!!

!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

In [7]:
def get_first_image_and_paragraph(page_url):
    response = requests.get(page_url)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    figure = soup.find('figure', {'typeof': 'mw:File/Thumb'})
    image_url = None
    if figure:
        img_tag = figure.find('img')
        if img_tag and 'src' in img_tag.attrs:
            image_url = 'https:' + img_tag['src']

    first_paragraph = None
    for p in soup.find_all('p'):
        if p.text.strip():
            for sup in p.find_all('sup'):
                sup.decompose()
            first_paragraph = ' '.join(p.text.strip().split())
            break

    return image_url, first_paragraph

def extract_images_and_paragraphs_from_file(filename):
    with open(filename, 'r') as file:
        html_content = file.read()
        soup = BeautifulSoup(html_content, 'html.parser')
        links = soup.find_all('a')

        details = []
        for link in links:
            href = link.get('href')
            if href and href.startswith('/wiki/') and not 'redlink=1' in href:
                full_url = 'https://en.wikipedia.org' + href
                image_url, description = get_first_image_and_paragraph(full_url)
                name = href[6:].replace('_', ' ')
                
                if image_url and description:
                    details.append((name, image_url, description))

    return details

def generate_sql_for_equipment(equipment_details, last_image_id, equipment_dict):
    sql_commands = []
    image_id = last_image_id 
    equipment_id = 1

    for name, image_url, manual in equipment_details:
        
        if name in equipment_dict.values() or name == 'Saut%C3%A9 pan':
            continue
            
        image_id += 1
        safe_manual = manual.replace('"', '\\"')
        #image_sql = f'INSERT INTO Image (image_url, description) VALUES ("{image_url}", "This is an image for {name}");'
        #equipment_sql = f'INSERT INTO Equipment (name, manual, image_id) VALUES ("{name}", "{safe_manual}", {image_id});'
        image_sql = f"""INSERT INTO Image (image_url, description) VALUES ("{image_url}", "This is an image for {name}");"""
        equipment_sql = f"""INSERT INTO Equipment (name, manual, image_id) VALUES ("{name}", "{safe_manual}", {image_id});"""
        
        sql_commands.append(image_sql)
        sql_commands.append(equipment_sql)
        
        equipment_dict[equipment_id] = name 
        equipment_id += 1  

    return image_id, sql_commands, equipment_dict


In [8]:
kitchenware_details = extract_images_and_paragraphs_from_file(file_path + 'kitchenware.txt')
cookware_details = extract_images_and_paragraphs_from_file(file_path + 'cookware.txt')

equipment_details = kitchenware_details + cookware_details

In [48]:
equipment_dict = {}
last_image_id, sql_commands, equipment_dict= generate_sql_for_equipment(equipment_details, last_image_id, equipment_dict)

with open(file_path + 'insert_data.sql', 'a', encoding='utf-8') as file:
    file.write('\n'.join(sql_commands))
    file.write('\n')

print("All insertion queries for equipment images and equipment details have been saved to 'insert_data.sql'.")

All insertion queries for equipment images and equipment details have been saved to 'insert_data.sql'.


!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

INGREDIENTS !!!!!!!!!!!!!

!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

In [10]:
food_df = file_path + 'Food.json'
base_url = "https://foodb.ca/system/foods/pictures/"
data = []

with open(food_df, 'r', encoding='utf-8') as file:
    for line in file:
        try:
            entry = json.loads(line)
            if 'id' in entry and 'name' in entry:
                data.append(entry)
        except json.JSONDecodeError:
            continue 
            
foods_df = pd.DataFrame(data)
if 'id' in foods_df.columns:
    foods_df['image_url'] = base_url + foods_df['id'].astype(str) + '/full/' + foods_df['id'].astype(str) + '.png'


for entry in data:
    food_id = entry['id']
    food_name = entry['name']
    image_url = f"{base_url}{food_id}/full/{food_id}.png"
    #print(f"Name: {food_name}, Image URL: {image_url}")


In [11]:
category_map = {
    'Herbs and Spices': 'Spices and essential oils',
    'Vegetables': 'Various foods of plant origin',
    'Fruits': 'Various foods of plant origin',
    'Nuts': 'Various foods of plant origin',
    'Cereals and cereal products': 'Cereals and their products',
    'Pulses': 'Various foods of plant origin',
    'Teas': 'Coffee, tea and their products',
    'Gourds': 'Various foods of plant origin',
    'Coffee and coffee products': 'Coffee, tea and their products',
    'Soy': 'Various foods of plant origin',
    'Cocoa and cocoa products': 'Various foods of plant origin',
    'Beverages': 'Various drinks',
    'Aquatic foods': 'Fish and their products',
    'Animal foods': 'Meat and its products',
    'Milk and milk products': 'Milk, eggs and their products',
    'Eggs': 'Milk, eggs and their products',
    'Confectioneries': 'Preserved foods',
    'Baking goods': 'Various foods of plant origin',
    'Dishes': 'Various foods of plant origin',
    'Snack foods': 'Various foods of plant origin',
    'Baby foods': 'Various foods of plant origin',
    'Fats and oils': 'Fats and oils',
    'Unclassified': 'Various foods of plant origin',
    'None': 'Various foods of plant origin'
}

foods_df['mapped_group'] = foods_df['food_group'].map(category_map)
foods_df['mapped_group'] = foods_df['mapped_group'].fillna('Various foods of plant origin')


#print(foods_df[['food_group', 'mapped_group']].head(30))


In [12]:
food_calories_df = pd.read_csv(file_path + 'food_calories.txt', sep='\t')
final_food_df = food_calories_df.copy()

name_to_id_dict = {v: k for k, v in food_group_dict.items()}

def find_best_match(food_item, search_df):
    # Full name match first
    matches = process.extractOne(food_item, search_df['name'], scorer=fuzz.token_sort_ratio)
    if matches[1] >= 90: 
        matched_row = search_df[search_df['name'] == matches[0]]
        return matched_row['image_url'].iloc[0], matched_row['mapped_group'].iloc[0]
    
    words = food_item.split()
    for word in words:
        matches = process.extractOne(word, search_df['name'], scorer=fuzz.token_sort_ratio)
        if matches[1] >= 70: 
            matched_row = search_df[search_df['name'] == matches[0]]
            if not matched_row.empty:
                return matched_row['image_url'].iloc[0], matched_row['mapped_group'].iloc[0]
    
    return "https://upload.wikimedia.org/wikipedia/commons/1/14/No_Image_Available.jpg", "Various foods of plant origin" 


final_food_df[['image_url', 'food_group']] = final_food_df['Food'].apply(
    lambda x: find_best_match(x, foods_df)
).apply(pd.Series) 

final_food_df['food_group'] = final_food_df['food_group'].map(name_to_id_dict)

#print(final_food_df.head(10))


In [49]:
def generate_ingredient_sql(df, last_image_id):
    sql_commands = []
    image_id = last_image_id 
    ingredient_id = 1 
    ingredient_dict = {}

    for idx, row in df.iterrows():
        safe_food = row['Food'].replace("'", " ")
        safe_food = safe_food.replace("’", " ")
        image_description = f"This is an image for {safe_food}"
        image_sql = f"""INSERT INTO Image (image_url, description) VALUES ('{row['image_url']}', '{image_description}');"""
        sql_commands.append(image_sql)
        
        avg_grams = calculate_avg_grams(row['Share']) 
        
        image_id += 1  
        ingredient_sql = f"""INSERT INTO Ingredient (
            name, kcal_per_100, image_id, food_group_id, avg_grams
        ) VALUES (
            '{safe_food}', {row['Calories/100g-ml']}, {image_id}, {row['food_group']}, {avg_grams}
        );"""
        sql_commands.append(ingredient_sql)

        ingredient_dict[safe_food] = ingredient_id
        ingredient_id += 1 
        
    return image_id, sql_commands, ingredient_dict

def calculate_avg_grams(share_description):
    
    default_grams = 100
    
    match = re.search(r'(\d+|[\d/]+) (cup|piece|fruit|boxes|slice|tsp)[s]* \((\d+)gr\)', share_description)
    if match:
        servings = match.group(1)
        grams = int(match.group(3))
        try:
            servings = float(Fraction(servings))
        except ValueError:
            servings = float(servings)
        return grams / servings if servings > 0 else grams

    match = re.search(r'(\d+)gr', share_description)
    if match:
        return int(match.group(1))

    match = re.search(r'(\d+)ml', share_description)
    if match:
        return int(match.group(1))
    
    return default_grams

image_id, sql_commands, ingredient_dict = generate_ingredient_sql(final_food_df, last_image_id)
last_image_id = image_id

with open(file_path + 'insert_data.sql', 'a', encoding='utf-8') as file:
    file.write('\n'.join(sql_commands))
    file.write('\n')

print("All insertion queries for ingredient images and ingredients have been saved to 'insert_data.sql'.")


All insertion queries for ingredient images and ingredients have been saved to 'insert_data.sql'.


!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

ADMIN !!!!!!!!!!!!!!!!!!!!!!!!!!!!!

!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

In [50]:
fake = Faker()

sql_commands = []

first_name = fake.first_name_male()
last_name = fake.last_name_male()
username = "chef_botakis"
password = "12345678"

admin_sql = f"""INSERT INTO Administrator (first_name, last_name, username, password) VALUES ('{first_name}', '{last_name}', '{username}', '{password}');"""
user_sql = f"""CREATE USER '{username}' IDENTIFIED BY '{password}';"""
role_sql = f"""GRANT 'Administrator' TO '{username}';"""
set_role = f"""SET DEFAULT ROLE "Administrator" FOR '{username}';"""
sql_commands.append(admin_sql)
sql_commands.append(user_sql)
sql_commands.append(role_sql)
sql_commands.append(set_role)

with open(file_path + 'insert_data.sql', 'a', encoding='utf-8') as file:
    file.write('\n'.join(sql_commands))
    file.write('\n')

print("Admin insertion queries have been saved to 'insert_data.sql'.")

Admin insertion queries have been saved to 'insert_data.sql'.


!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

COOKS !!!!!!!!!!!!!!!!!!!!!!!!!!!!!

!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

In [51]:
male_img_base_url = "https://xsgames.co/randomusers/assets/avatars/male/"
female_img_base_url = "https://xsgames.co/randomusers/assets/avatars/female/"

fake = Faker()

def determine_ranking(years):
    if years > 20:
        return 'chef'
    elif years > 15:
        return 'sous chef'
    elif years > 10:
        return 'cook A'
    elif years > 5:
        return 'cook B'
    else:
        return 'cook C'


def generate_cooks(num_males, num_females, male_img_base_url, female_img_base_url, last_image_id):
    sql_commands = []
    image_id = last_image_id
    
    # create one dummy user for testing
    first_name = fake.first_name_male()
    last_name = fake.last_name_male()
    contact_number = fake.phone_number()
    date_of_birth = fake.date_of_birth(minimum_age=18, maximum_age=65).isoformat()
    years_of_experience = random.randint(0, 30)
    ranking = determine_ranking(years_of_experience)
    username = "aplos_botakis"
    password = "12345678"
        
    image_url = f"{male_img_base_url}1.jpg"
    image_description = f"This is an image for cook {first_name} {last_name}"
    image_sql = f"INSERT INTO Image (image_url, description) VALUES ('{image_url}', '{image_description}');"
    sql_commands.append(image_sql)

    cook_sql = f"""INSERT INTO Cook (first_name, last_name, contact_number, date_of_birth, years_of_experience, ranking, username, password, image_id) VALUES
    ('{first_name}', '{last_name}', '{contact_number}', '{date_of_birth}', {years_of_experience}, '{ranking}', '{username}', '{password}', {image_id});"""
    sql_commands.append(cook_sql)
        
    user_sql = f"""CREATE USER '{username}' IDENTIFIED BY '{password}';"""
    role_sql = f"""GRANT 'Cook_User' TO '{username}';"""
    set_role = f"""SET DEFAULT ROLE "Cook_User" FOR '{username}';"""
    sql_commands.append(user_sql)
    sql_commands.append(role_sql)
    sql_commands.append(set_role)
        
    image_id += 1

    for i in range(2, num_males + 1):
        first_name = fake.first_name_male()
        last_name = fake.last_name_male()
        contact_number = fake.phone_number()
        date_of_birth = fake.date_of_birth(minimum_age=18, maximum_age=65).isoformat()
        years_of_experience = random.randint(0, 30)
        ranking = determine_ranking(years_of_experience)
        username = f"{first_name.lower()}{last_name.lower()}{random.randint(10, 99)}"
        password = fake.password(length=10, special_chars=True, digits=True, upper_case=True, lower_case=True)
        
        image_url = f"{male_img_base_url}{i}.jpg"
        image_description = f"This is an image for cook {first_name} {last_name}"
        image_sql = f"INSERT INTO Image (image_url, description) VALUES ('{image_url}', '{image_description}');"
        sql_commands.append(image_sql)

        cook_sql = f"""INSERT INTO Cook (first_name, last_name, contact_number, date_of_birth, years_of_experience, ranking, username, password, image_id) VALUES
        ('{first_name}', '{last_name}', '{contact_number}', '{date_of_birth}', {years_of_experience}, '{ranking}', '{username}', '{password}', {image_id});"""
        sql_commands.append(cook_sql)
        
        user_sql = f"""CREATE USER '{username}' IDENTIFIED BY '{password}';"""
        role_sql = f"""GRANT 'Cook_User' TO '{username}';"""
        set_role = f"""SET DEFAULT ROLE "Cook_User" FOR '{username}';"""
        sql_commands.append(user_sql)
        sql_commands.append(role_sql)
        sql_commands.append(set_role)
        
        image_id += 1

    for i in range(1, num_females + 1):
        first_name = fake.first_name_female()
        last_name = fake.last_name_female()
        contact_number = fake.msisdn()[-10:]
        date_of_birth = fake.date_of_birth(minimum_age=18, maximum_age=65).isoformat()
        years_of_experience = random.randint(0, 30)
        ranking = determine_ranking(years_of_experience)
        username = f"{first_name.lower()}{last_name.lower()}{random.randint(10, 99)}"
        password = fake.password(length=10, special_chars=True, digits=True, upper_case=True, lower_case=True)
        
        image_url = f"{female_img_base_url}{i}.jpg"
        image_description = f"""This is an image for cook {first_name} {last_name}"""
        image_sql = f"""INSERT INTO Image (image_url, description) VALUES ('{image_url}', '{image_description}');"""
        sql_commands.append(image_sql)

        cook_sql = f"""INSERT INTO Cook (first_name, last_name, contact_number, date_of_birth, years_of_experience, ranking, username, password, image_id) VALUES
        ('{first_name}', '{last_name}', '{contact_number}', '{date_of_birth}', {years_of_experience}, '{ranking}', '{username}', '{password}', {image_id});"""
        sql_commands.append(cook_sql)
        
        user_sql = f"""CREATE USER '{username}' IDENTIFIED BY '{password}';"""
        role_sql = f"""GRANT 'Cook_User' TO '{username}';"""
        set_role = f"""SET DEFAULT ROLE "Cook_User" FOR '{username}';"""
        sql_commands.append(user_sql)
        sql_commands.append(role_sql)
        sql_commands.append(set_role)
        
        image_id += 1

    return sql_commands, image_id

sql_cooks, image_id = generate_cooks(50, 50, male_img_base_url, female_img_base_url, last_image_id)
total_cooks = 100
last_image_id = image_id
with open(file_path + 'insert_data.sql', 'a', encoding='utf-8') as file:
    file.write('\n'.join(sql_cooks))
    file.write('\n')

print("Cook insertion queries have been saved to 'insert_data.sql'.")

Cook insertion queries have been saved to 'insert_data.sql'.


!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

TOPICS !!!!!!!!!!!!!!!!!!!!!!!!!!!!!

!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

In [52]:
topics = [
    ("Village Recipes", "Traditional recipes from the village.", "https://www.yummytummyaarthi.com/wp-content/uploads/2016/04/1-9.jpg"),
    ("Risotto Recipes", "Creamy and comforting risotto dishes.", "https://hips.hearstapps.com/del.h-cdn.co/assets/17/35/1600x1600/square-1504128527-delish-mushroom-risotto.jpg?resize=1200:*"),
    ("Easter Sweets", "Delicious sweets perfect for Easter.", "https://www.readyseteat.com/sites/g/files/qyyrlu501/files/uploadedImages/img_9708_89897.jpg"),
    ("Summer Salads", "Refreshing salads for hot days.", "https://images.immediate.co.uk/production/volatile/sites/30/2022/06/Epic-summer-salad-3aeb697.jpg"),
    ("Winter Soups", "Hearty soups to warm you up.", "https://images.immediate.co.uk/production/volatile/sites/2/2017/10/Mushroom-Soup-2df69df.jpg?quality=90&resize=556,505"),
    ("Vegan Dishes", "Tasty vegan recipes for all.", "https://images.immediate.co.uk/production/volatile/sites/30/2023/01/Ponzu-tofu-poke-bowl-8733c67.jpg?quality=90&resize=440,400"),
    ("Quick Snacks", "Quick and easy snacks on the go.", "https://static.toiimg.com/photo/65163740.cms"),
    ("Holiday Meals", "Festive meals for holiday celebrations.", "https://shop.momofuku.com/cdn/shop/articles/Holidays_2021_Table_Spread_3.jpg?v=1636317855"),
    ("Dessert Ideas", "Creative desserts to impress.", "https://www.tasteofhome.com/wp-content/uploads/2018/01/exps21585_THCA153054D10_15_4b.jpg"),
    ("Healthy Smoothies", "Nutritious smoothies for a health boost.", "https://www.eatingwell.com/thmb/CokPYaf2YPnPACHBls_LVhyUp0g=/1500x0/filters:no_upscale():max_bytes(150000):strip_icc()/healthy-breakfast-smoothie-8029983-4000x4000-3e02d40929c8410c877a171a235c99bc.jpg"),
    ("BBQ Grilling", "Best recipes for BBQ grilling.", "https://fnsharp.com/cdn/shop/articles/backyard-bbq-meat-grilling-guide-850x600_850x.jpg?v=1617738809"),
    ("Seafood Specials", "Fresh and delicious seafood dishes.", "https://www.homemadeitaliancooking.com/wp-content/uploads/2018/12/plated_cioppino.jpg"),
    ("Pasta Favorites", "Classic pasta recipes loved by all.", "https://www.foodandwine.com/thmb/tAS-x_IC4ss1cb9EdDpsr0UExdM=/1500x0/filters:no_upscale():max_bytes(150000):strip_icc()/bucatini-with-mushroom-ragu-dandelion-greens-and-tarragon-FT-RECIPE0421-3a5f0d29f7264f5e9952d4a3a51f5f58.jpg"),
    ("Breakfast Options", "Start your day right with these breakfasts.", "https://www.eatingwell.com/thmb/t1T6R6ZAyG6j8-39Bdmh6eAr0F4=/1500x0/filters:no_upscale():max_bytes(150000):strip_icc()/egg-tartine-892cac2657114696ab634792d0bdcb4b.jpg"),
    ("Comfort Food", "Food that feels like a hug.", "https://www.allrecipes.com/thmb/4h5yzzM5u5_Pgx6zoazspPPCZJg=/1500x0/filters:no_upscale():max_bytes(150000):strip_icc()/237311-Slow-cooker-mac-and-cheese-ddmfs-1X1-1297-1-d5968b9dc757411fb6a973d37b311166.jpg"),
    ("Street Food Adventures", "Bold flavors from around the world.", "https://afar.brightspotcdn.com/dims4/default/5d22728/2147483647/strip/false/crop/2500x1667+0+0/resize/1486x991!/quality/90/?url=https%3A%2F%2Fafar-media-production-web.s3.us-west-2.amazonaws.com%2Fbrightspot%2Fda%2Ff3%2F1894b566a3721db50ec473fcf399%2Foriginal-shutterstock-1294137358.jpg"),
    ("Gluten-Free Goodies", "Delicious recipes without gluten.", "https://www.eatthis.com/wp-content/uploads/sites/4/2020/08/weldon-owen-gluten-free-cover.jpg?quality=82&strip=1"),
    ("Low-Carb Meals", "Tasty low-carb options for healthy eating.", "https://www.eatingwell.com/thmb/XF2id3sTOpQwtVx0A6Z_y7cHqdI=/1500x0/filters:no_upscale():max_bytes(150000):strip_icc()/cheesy-portobello-chicken-cutlets-with-broccoli-ae1449c758834bb7ac75437e37a14065.jpg"),
    ("Midnight Munchies", "Perfect for late-night cravings.", "https://img.cdn4dd.com/cdn-cgi/image/fit=cover,width=600,height=400,format=auto,quality=80/https://doordash-static.s3.amazonaws.com/media/store/header/6e298dc7-f46e-4ae8-8d6d-91722c0d7ea4.JPG")
]

def generate_topic_sql(topics, last_image_id, topic_dict):
    sql_commands = []
    image_id = last_image_id
    topic_id = 1
    
    for name, description, image_url in topics:
        image_description = f"This is a theme image for {name}"
        
        image_sql = f"INSERT INTO Image (image_url, description) VALUES ('{image_url}', '{image_description}');"
        sql_commands.append(image_sql)
        
        topic_sql = f"""INSERT INTO Topic (name, description, image_id) VALUES
        ('{name}', '{description}', {image_id});"""
        sql_commands.append(topic_sql)
        
        topic_dict[topic_id] = name
        topic_id += 1
        image_id += 1

    return sql_commands, image_id, topic_dict


topic_dict = {}
sql_topic_commands, new_image_id, topic_dict = generate_topic_sql(topics, last_image_id, topic_dict)
last_image_id = new_image_id

with open(file_path + 'insert_data.sql', 'a', encoding='utf-8') as file:
    file.write('\n'.join(sql_topic_commands))
    file.write('\n')

print("Topic insertion queries have been saved to 'insert_data.sql'.")


Topic insertion queries have been saved to 'insert_data.sql'.


!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!


RECIPES !!!!!!!!!!!!!!!!!!!!!!!!!!!!!


!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

This function gets the url from the small json file (that has less info) and gets the full data from the database for a specific recipe based on its url (that can be found on the small url)

In [19]:
def scrape_recipe(url):
    try:
        response = requests.get(url)
        response.raise_for_status() 
        soup = BeautifulSoup(response.text, 'html.parser')

        scripts = soup.find_all('script', type='application/ld+json')
        for script in scripts:
            try:
                data = json.loads(script.string)

                if '@type' in data and data['@type'] == 'Recipe':
                    return data
            except json.JSONDecodeError:
                continue
    except requests.RequestException as e:
        print(f"Error fetching {url}: {e}")
    return None

In [20]:
def parse_duration(duration):
    """ Parse ISO 8601 duration into minutes. """
    match = re.match(r'P(?:(\d+)D)?T(?:(\d+)H)?(?:(\d+)M)?', duration)
    days = int(match.group(1) or 0)
    hours = int(match.group(2) or 0)
    minutes = int(match.group(3) or 0)
    return days * 1440 + hours * 60 + minutes

def infer_recipe_type(description):
    """ Infer recipe type based on description or ingredients. """
    keywords = {'pastry': ['flour', 'bake', 'oven'], 'cooking': ['fry', 'boil', 'grill']}
    for type_, clues in keywords.items():
        if any(clue in description.lower() for clue in clues):
            return type_
    return 'cooking' 

def infer_difficulty(preptime, cooktime):
    """ Infer difficulty based on preparation and cooking times on a scale from 1 to 5. """
    total_time = preptime + cooktime
    if total_time <= 20:
        return 1 
    elif total_time <= 40:
        return 2 
    elif total_time <= 60:
        return 3 
    elif total_time <= 120:
        return 4 
    else:
        return 5  

def extract_numbers(text):
    """ Extract the first number from a string. """
    numbers = re.findall(r'\d+', text)
    return int(numbers[0]) if numbers else 1 

def random_nutritional_value(min_value, max_value):
    """ Generate a random float within given range, formatted to two decimal places. """
    return round(random.uniform(min_value, max_value), 2)

In [21]:
def get_nationality_id_from_cuisine(cuisine, nationality_dict):
    if cuisine:
        matches = process.extractOne(cuisine, list(nationality_dict.values()), score_cutoff=80) #80
        if matches:
            return next(key for key, value in nationality_dict.items() if value == matches[0])
    return random.randint(1, len(nationality_dict))

In [22]:
def preprocess_ingredient(ingredient_name):

    qualifiers = [
        'fresh', 'chopped', 'diced', 'sliced', 'ground', 'crushed', 'canned',
        'tsp', 'tbsp', 'cup', 'cups', 'ml', 'g', 'small', 'medium', 'large',
        'minced', 'peeled', 'washed', 'dried'
    ]
    words = ingredient_name.split()
    filtered_words = [word for word in words if word.lower() not in qualifiers]
    return ' '.join(filtered_words)

def find_closest_food_match(ingredient_name, food_df):
    processed_ingredient = preprocess_ingredient(ingredient_name)
    

    matches = process.extract(processed_ingredient, food_df['Food'], scorer=fuzz.token_set_ratio, limit=3)

    for match, score, _ in matches:  
        if score >= 90:
            if 'medium' in processed_ingredient and 'medium' not in match:
                continue 
            return match  

    if matches:
        return matches[0][0]  
    
    return "No close match found"


In [23]:
def parse_ingredient(ingredient):
    
    fraction_dict = {
        '½': 0.5, '⅓': 1/3, '¼': 0.25, '⅔': 2/3,
        '¾': 0.75, '⅛': 0.125, '⅜': 0.375,
        '⅝': 0.625, '⅞': 0.875
    }
    non_numeric_words = ["some", "a lot of", "much", "enough", "a pinch of", "few"]

    ingredient = ingredient.split(',')[0].strip()
    ingredient = re.sub(r'oz|/lb|/1lb|/fl', '', ingredient).strip()

    quantity_unit_pattern = r"(\d+\.?\d*|\d*\s*\d/\d|½|⅓|¼|⅔|¾|⅛|⅜|⅝|⅞)\s*(ml|g\b|gr\b)?"
    match = re.search(quantity_unit_pattern, ingredient, re.IGNORECASE)
    
    serving_type = None

    if match:
        quantity = match.group(1)
        unit = match.group(2)

        if quantity in fraction_dict:
            quantity = fraction_dict[quantity]
        else:
            try:
                quantity = float(Fraction(quantity))
            except (ValueError, TypeError):
                quantity = None  

        remaining_text = re.sub(quantity_unit_pattern, '', ingredient).strip()
        possible_units = ['cups', 'cup', 'tbsp', 'tsp', 'pinch', 'cloves', 'clove']
        for possible_unit in possible_units:
            if possible_unit in remaining_text.split():
                serving_type = possible_unit
                break

        if unit in ['g', 'ml']:
            quantity_type = 'grams'
        elif unit or serving_type:
            quantity_type = 'serving'
            serving_type = unit or serving_type
        else:
            quantity_type = 'serving'

        ingredient_name = remaining_text
    else:
        quantity = random.choice(non_numeric_words)
        quantity_type = 'non_numeric'
        ingredient_name = ingredient.strip()

    return ingredient_name, quantity_type, quantity, serving_type

# Example usage
example_ingredient = "3 ½ cups flour"
ingredient_name, quantity_type, quantity, serving_type = parse_ingredient(example_ingredient)
print(f'Ingredient: {ingredient_name}, Quantity Type: {quantity_type}, Quantity: {quantity}, Serving Type: {serving_type}')


Ingredient: cups flour, Quantity Type: serving, Quantity: 3.0, Serving Type: cups


In [24]:
recipes = []
image_id = last_image_id
with open(file_path + 'recipeitems-latest.json', 'r', encoding='utf-8') as file:
    while len(recipes) < 100:
        line = file.readline()
        if not line:
            break
        try:
            recipe_meta = json.loads(line)
            if (not recipe_meta.get('image')) or recipe_meta.get('image').startswith("http://static.thepioneerwoman.com") or recipe_meta.get('image').startswith("http://cdn.naturallyella.com"):
                continue
            url = recipe_meta.get('url') 
            if url:
                if url.startswith("http://www.bonappetit.com") or url.startswith("https://www.aspicyperspective.com") or url.startswith("http://delishhh.com"):
                    continue
                recipe = scrape_recipe(url)
                if recipe:
                    recipe['image'] = recipe_meta.get('image')
                    recipe['yield'] = recipe_meta.get('image')
                    if not all([recipe.get('name'), recipe.get('description'), recipe.get('image'), recipe.get('prepTime'), recipe.get('cookTime'), recipe.get('recipeInstructions'), recipe.get('recipeIngredient'), recipe.get('recipeCuisine'), recipe.get('recipeCategory'), recipe.get('recipeYield')]):
                        continue
                    recipes.append(recipe)

        except json.JSONDecodeError:
            continue


Error fetching http://www.seriouseats.com/recipes/2008/11/the-slope-recipe-drink-brooklyn-clover-club.html: 406 Client Error: Not Acceptable for url: https://www.seriouseats.com/recipes/2008/11/the-slope-recipe-drink-brooklyn-clover-club.html
Error fetching http://www.seriouseats.com/recipes/2008/02/cocktails-red-hook-recipe.html: 405 Client Error: Signal - Not Acceptable for url: http://www.seriouseats.com/recipes/2008/02/cocktails-red-hook-recipe.html
Error fetching http://www.seriouseats.com/recipes/2007/12/dinner-tonight-balsamic-glazed-brussels-sprouts-recipe.html: 405 Client Error: Signal - Not Acceptable for url: http://www.seriouseats.com/recipes/2007/12/dinner-tonight-balsamic-glazed-brussels-sprouts-recipe.html
Error fetching http://www.seriouseats.com/recipes/2008/02/cocktails-widows-kiss-recipe.html: 405 Client Error: Signal - Not Acceptable for url: http://www.seriouseats.com/recipes/2008/02/cocktails-widows-kiss-recipe.html
Error fetching http://www.seriouseats.com/recipe

In [53]:
def generate_sql_and_dict(entries, table_name):
    sql_commands = []
    entry_dict = {}
    for idx, entry in enumerate(entries, start=1):
        sql_commands.append(f"INSERT INTO {table_name} (name) VALUES ('{entry}');")
        entry_dict[idx] = entry
    return sql_commands, len(entries), entry_dict

# Meal types
meal_types = [
    'breakfast', 'brunch', 'second breakfast', 'elevenses', 'lunch', 'tea',
    'dinner', 'supper', 'late-night snack', 'midnight meal', 'holiday feast',
    'weekend dinner', 'light snack', 'afternoon snack', 'pre-workout meal'
]

# Tags
tags = [
    'quick-lunch', 'cold-dish', 'vegetarian', 'low-carb', 'family-friendly', 'quick', 'healthy',
    'gluten-free', 'vegan', 'no-cook', 'make-ahead', 'one-pot', 'summer', 'winter', 'festive',
    'comfort food', 'low-fat', 'high-protein', 'kid-friendly', 'budget-friendly', 'party', 'gourmet',
    'starters & nibbles', 'other', 'desserts', 'main course', 'light meals & snacks',
    'cakes and baking', 'side dishes', 'brunch'
]

# Tips
tips = [
    "Can be stored in the refrigerator for up to 3 days.",
    "Best served hot.",
    "Can be frozen for one month.",
    "Prep the night before for quicker assembly.",
    "Adjust spices to taste.",
    "Add a splash of lemon for extra zest.",
    "Keep hydrated while cooking.",
    "Taste as you go.",
    "Use fresh herbs for better flavor.",
    "Clean as you cook to save time.",
    "Marinate overnight for deeper flavor.",
    "Serve with crusty bread to complement the dish.",
    "Perfect for batch cooking.",
    "Garnish with fresh greens for added color.",
    "A pinch of sugar can balance acidity."
]

meal_type_sql, meal_type_count, meal_type_dict = generate_sql_and_dict(meal_types, "Meal_type")
tag_sql, tag_count, tags_dict = generate_sql_and_dict(tags, "Tag")
tip_sql, tip_count, tips_dict = generate_sql_and_dict(tips, "Tip")

# Writing to SQL file
with open(file_path + 'insert_data.sql', 'a', encoding='utf-8') as file:
    file.write('\n'.join(meal_type_sql))
    file.write('\n')
    file.write('\n'.join(tag_sql))
    file.write('\n')
    file.write('\n'.join(tip_sql))
    file.write('\n')

print(f"Total Meal Types: {meal_type_count}")
print(f"Total Tags: {tag_count}")
print(f"Total Tips: {tip_count}")

Total Meal Types: 15
Total Tags: 30
Total Tips: 15


In [54]:
nationality_cook_dict = {}

nationality_ids_from_recipes = set()
for recipe in recipes:
    nationality_id = get_nationality_id_from_cuisine(recipe['recipeCuisine'], nationality_dict)
    nationality_ids_from_recipes.add(nationality_id)

all_nationality_ids = set(nationality_dict.keys())

additional_nationality_ids = list(all_nationality_ids - nationality_ids_from_recipes)

sql_commands = []
#total_cooks = 100 from cooks generation
cook_ids = range(1, total_cooks + 1)  

for cook_id in cook_ids:
    mandatory_nationalities = random.sample(nationality_ids_from_recipes, random.randint(4, 7))
    nationality_cook_dict[cook_id] = mandatory_nationalities

    additional_nationalities = random.sample(additional_nationality_ids, random.randint(4, 9))
    total_nationalities = mandatory_nationalities + additional_nationalities

    for nationality_id in total_nationalities:
        sql_command = f"""INSERT INTO Nationality_Cook (nationality_id, cook_id) VALUES ({nationality_id}, {cook_id});"""
        sql_commands.append(sql_command)

with open(file_path + 'insert_data.sql', 'a', encoding='utf-8') as file:
    file.write('\n'.join(sql_commands))
    file.write('\n')

print("Nationality_Cook insertion queries have been saved.")

Nationality_Cook insertion queries have been saved.


C:\Users\nikol\AppData\Local\Temp\ipykernel_6768\2178677979.py:17: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  mandatory_nationalities = random.sample(nationality_ids_from_recipes, random.randint(4, 7))


In [55]:
sql_commands = []
recipe_id = 1
image_id = last_image_id
recipe_nationality_dict = {}

for recipe in recipes:
    name = recipe['name'].replace("'", "''")
    description = recipe['description'].replace("'", "''")
    description = description.replace("’", "''")
    image_url = recipe['image']
    prep_time = parse_duration(recipe.get('prepTime', 'PT0M'))
    cook_time = parse_duration(recipe.get('cookTime', 'PT0M'))
    portions = extract_numbers(recipe.get('recipeYield', '1'))
    difficulty = infer_difficulty(prep_time, cook_time)
    ingredients = recipe['recipeIngredient']
    steps = recipe['recipeInstructions']
    nationality = recipe['recipeCuisine']
    category = recipe['recipeCategory']
    
    if 'Cakes and baking' == category or 'Desserts' == category:
        recipe_type = 'pastry'
    else:
        recipe_type = 'cooking'

    fat_grams_per_portion = random_nutritional_value(5, 30)
    protein_grams_per_portion = random_nutritional_value(10, 50)
    curbs_grams_per_portion = random_nutritional_value(20, 100)
    kcal_per_portion = '0.00'
    
    image_id += 1
    image_description = f"This is an image for {name}."
    image_sql = f"INSERT INTO Image (image_url, description) VALUES ('{image_url}', '{image_description}');"
    sql_commands.append(image_sql)
    
    nationality_id = get_nationality_id_from_cuisine(recipe['recipeCuisine'], nationality_dict)
    recipe_nationality_dict[recipe_id] = nationality_id
    sql = f"""INSERT INTO Recipe (recipe_type, difficulty, name, description, preparation_time, execution_time, portions, fat_grams_per_portion, protein_grams_per_portion, curbs_grams_per_portion, kcal_per_portion, image_id, nationality_id) VALUES ('{recipe_type}', {difficulty}, '{name}', '{description}', {prep_time}, {cook_time}, {portions}, {fat_grams_per_portion}, {protein_grams_per_portion}, {curbs_grams_per_portion}, {kcal_per_portion}, {image_id}, {nationality_id});"""
    sql_commands.append(sql)
    
    basic_ingredient_selected = False
    used_ingredient_ids = set()
    for ingredient in ingredients:
        ingredient_name, quantity_type, quantity, serving_type = parse_ingredient(ingredient)
        matched_ingredient = find_closest_food_match(ingredient_name, final_food_df)
        ingredient_id = ingredient_dict.get(matched_ingredient)
        
        if ingredient_id in used_ingredient_ids:
            continue

        used_ingredient_ids.add(ingredient_id)

        if ingredient_id:
            basic_ingredient = 'FALSE'
            if not basic_ingredient_selected:
                basic_ingredient = 'TRUE'
                basic_ingredient_selected = True
            
            ingredient_sql = f"""INSERT INTO Recipe_Ingredient (recipe_id, ingredient_id, basic_ingredient, quantity_type, quantity, serving_type) VALUES ({recipe_id}, {ingredient_id}, {basic_ingredient}, '{quantity_type}', '{quantity}', '{serving_type}');"""
            sql_commands.append(ingredient_sql)
            
            
    num_equipment = random.randint(5, 8)
    chosen_equipment_ids = random.sample(list(equipment_dict.keys()), num_equipment)
    for equipment_id in chosen_equipment_ids:
        equipment_sql = f"""INSERT INTO Recipe_Equipment (recipe_id, equipment_id) VALUES ({recipe_id}, {equipment_id});"""
        sql_commands.append(equipment_sql)
        
    num_topic = random.randint(1, 3)
    chosen_topic_ids = random.sample(list(topic_dict.keys()), num_topic)
    for topic_id in chosen_topic_ids:
        topic_sql = f"""INSERT INTO Recipe_Topic (recipe_id, topic_id) VALUES ({recipe_id}, {topic_id});"""
        sql_commands.append(topic_sql)
    
    sequence = 1  
    for step in steps:
        step_description = step.replace("'", "''")
        step_description = step_description.replace("’", "''")
        step_sql = f"""INSERT INTO Step (recipe_id, name, sequence) VALUES ({recipe_id}, '{step_description}', {sequence});"""
        sql_commands.append(step_sql)
        sequence += 1 
        
    assigned_meal_type_ids = random.sample(range(1, meal_type_count + 1), random.randint(1, 3))
    for meal_type_id in assigned_meal_type_ids:
        meal_type_sql = f"INSERT INTO Recipe_Meal_Type (recipe_id, meal_type_id) VALUES ({recipe_id}, {meal_type_id});"
        sql_commands.append(meal_type_sql)

    category_id = None
    for tag_id, tag_name in tags_dict.items():
        if tag_name == category.lower():
            category_id = tag_id
            break
    
    if category_id is None:
        assigned_tag_ids = random.sample(range(1, tag_count + 1), random.randint(1, 3))
    else:
        available_tag_ids = [tid for tid in range(1, tag_count + 1) if tid != category_id]
        additional_tag_ids = random.sample(available_tag_ids, random.randint(1, 3))
        assigned_tag_ids = [category_id] + additional_tag_ids
            
    for tag_id in assigned_tag_ids:
        tag_sql = f"INSERT INTO Recipe_Tag (recipe_id, tag_id) VALUES ({recipe_id}, {tag_id});"
        sql_commands.append(tag_sql)

    assigned_tip_ids = random.sample(range(1, tip_count + 1), random.randint(1, 3))
    for tip_id in assigned_tip_ids:
        tip_sql = f"INSERT INTO Recipe_Tip (recipe_id, tip_id) VALUES ({recipe_id}, {tip_id});"
        sql_commands.append(tip_sql)
        
    recipe_id += 1  

last_image_id = image_id
last_recipe_id = recipe_id - 1

with open(file_path + 'insert_data.sql', 'a', encoding='utf-8') as file:
    file.write('\n'.join(sql_commands))
    file.write('\n')

print("SQL generation complete. All insertion queries have been saved to 'insert_data.sql'.")


SQL generation complete. All insertion queries have been saved to 'insert_data.sql'.


In [56]:
#last_recipe_id = 100
all_recipe_ids = list(range(1, last_recipe_id + 1))

sql_commands = []

for cook_id, nationalities in nationality_cook_dict.items():

    recipes_for_cook = []

    for recipe_id, recipe_nationality in recipe_nationality_dict.items():
        if recipe_nationality in nationalities:
            sql_command = f"""INSERT INTO Recipe_Cook (recipe_id, cook_id) VALUES ({recipe_id}, {cook_id});"""
            sql_commands.append(sql_command)
            recipes_for_cook.append(recipe_id)

    available_recipes = list(set(all_recipe_ids) - set(recipes_for_cook))

    if len(available_recipes) >= 10:
        random_additional_recipes = random.sample(available_recipes, 10)
    else:
        random_additional_recipes = available_recipes

    for additional_recipe_id in random_additional_recipes:
        sql_command = f"""INSERT INTO Recipe_Cook (recipe_id, cook_id) VALUES ({additional_recipe_id}, {cook_id});"""
        sql_commands.append(sql_command)

with open(file_path + 'insert_data.sql', 'a', encoding='utf-8') as file:
    file.write('\n'.join(sql_commands))
    file.write('\n')

print("Recipe_Cook insertion queries, including additional random recipes, have been saved.")

Recipe_Cook insertion queries, including additional random recipes, have been saved.
